In [ ]:
!pip install split-folders

In [ ]:
input_folder = '/content/drive/MyDrive/Trimmed_AID'

In [ ]:
import splitfolders as sf

In [ ]:
sf.ratio(input_folder,output = '/content/drive/MyDrive/Trimmed_AID_dataset',seed=42,ratio=(.7,.2,.1),group_prefix=None)

Copying files: 3369 files [01:24, 39.74 files/s]


Start


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout


In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
    directory='/content/drive/MyDrive/Trimmed_AID_dataset/train',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))
validation_ds = keras.utils.image_dataset_from_directory(
    directory='/content/drive/MyDrive/Trimmed_AID_dataset/val',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))
test_ds = keras.utils.image_dataset_from_directory(
    directory='/content/drive/MyDrive/Trimmed_AID_dataset/test',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256))

Found 1346 files belonging to 30 classes.
Found 672 files belonging to 30 classes.
Found 1351 files belonging to 30 classes.


In [ ]:
classes = ['Airport', 'BareLand', 'BaseballField', 'Beach', 'Bridge', 'Center', 'Church', 'Commercial', 'DenseResidential', 'Desert', 'Farmland', 'Forest', 'Industrial', 'Meadow', 'MediumResidential', 'Mountain', 'Park', 'Parking', 'Playground', 'Pond', 'Port', 'RailwayStation', 'Resort', 'River', 'School', 'SparseResidential', 'Square', 'Stadium', 'StorageTanks', 'Viaduct']
num_classes = len(classes)

ResNet

In [ ]:
from keras.applications import ResNet50

In [ ]:
# Build ResNet model
resnet_model = Sequential()
resnet_model.add(ResNet50(include_top=False, weights='imagenet', input_shape=(256, 256, 3)))
resnet_model.add(Flatten())
resnet_model.add(Dense(256, activation='relu'))
resnet_model.add(Dropout(0.5))
resnet_model.add(Dense(num_classes, activation='softmax'))

# Compile and fit ResNet model
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.fit(train_ds, epochs=10, validation_data=validation_ds)

94765736/94765736 [==============================] - 4s 0us/step
Epoch 1/10
43/43 [==============================] - 740s 14s/step - loss: 7.9304 - accuracy: 0.1738 - val_loss: 248244.4531 - val_accuracy: 0.0298
Epoch 2/10
43/43 [==============================] - 26s 557ms/step - loss: 2.8830 - accuracy: 0.2689 - val_loss: 40.8276 - val_accuracy: 0.0565
Epoch 3/10
43/43 [==============================] - 29s 632ms/step - loss: 2.3111 - accuracy: 0.3945 - val_loss: 41.1628 - val_accuracy: 0.0565
Epoch 4/10
43/43 [==============================] - 25s 551ms/step - loss: 1.9117 - accuracy: 0.4926 - val_loss: 11.3198 - val_accuracy: 0.0565
Epoch 5/10
43/43 [==============================] - 29s 631ms/step - loss: 1.7596 - accuracy: 0.5817 - val_loss: 33.1067 - val_accuracy: 0.0298
Epoch 6/10
43/43 [==============================] - 29s 636ms/step - loss: 1.3490 - accuracy: 0.6657 - val_loss: 12.3297 - val_accuracy: 0.0298
Epoch 7/10
43/43 [==============================] - 25s 538ms/step -

there seem to be some issues with the validation loss and accuracy. The validation loss is extremely high, which indicates that the model might not be generalizing well to unseen data. Additionally, the validation accuracy is very low, which further confirms the poor performance on the validation set.

In [ ]:
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

Regularization

let's add a few more regularization techniques to our model. Apart from dropout, we can include batch normalization and L2 regularization.

We added an L2 regularization term to the Dense layer with a regularization strength of 0.001. This helps prevent overfitting by penalizing large weights.

We included a BatchNormalization layer after the Dense layer. Batch normalization normalizes the activations of the previous layer, which can help stabilize and speed up the training process.

In [ ]:
# Build ResNet model
resnet_model = Sequential()
resnet_model.add(ResNet50(include_top=False, weights='imagenet', input_shape=(256, 256, 3)))
resnet_model.add(Flatten())
resnet_model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))  # L2 regularization
resnet_model.add(BatchNormalization())  # Batch normalization layer
resnet_model.add(Dropout(0.5))  # Dropout layer
resnet_model.add(Dense(num_classes, activation='softmax'))


# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# Compile and fit ResNet model with callbacks
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.fit(train_ds, epochs=10, validation_data=validation_ds, callbacks=[early_stopping, checkpoint])

Epoch 1/10
43/43 [==============================] - 66s 819ms/step - loss: 3.7201 - accuracy: 0.3596 - val_loss: 868.4080 - val_accuracy: 0.0298
Epoch 2/10
43/43 [==============================] - 35s 763ms/step - loss: 2.7750 - accuracy: 0.6315 - val_loss: 20.1528 - val_accuracy: 0.0565
Epoch 3/10
43/43 [==============================] - 29s 639ms/step - loss: 2.1139 - accuracy: 0.8046 - val_loss: 49.3502 - val_accuracy: 0.0298
Epoch 4/10
43/43 [==============================] - 29s 646ms/step - loss: 1.9483 - accuracy: 0.8432 - val_loss: 9.2003 - val_accuracy: 0.0298
Epoch 5/10
43/43 [==============================] - 29s 634ms/step - loss: 1.7723 - accuracy: 0.8997 - val_loss: 10.9683 - val_accuracy: 0.0565
Epoch 6/10
43/43 [==============================] - 29s 636ms/step - loss: 2.2845 - accuracy: 0.7875 - val_loss: 339.0048 - val_accuracy: 0.0551
Epoch 7/10
43/43 [==============================] - 26s 559ms/step - loss: 3.4590 - accuracy: 0.5617 - val_loss: 1377.1833 - val_accura

Data Preprocessing:

Ensuring that your data is properly preprocessed. This typically involves normalizing the pixel values to be between 0 and 1 and any other preprocessing techniques such as data augmentation.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define ImageDataGenerator for preprocessing with data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 32 using train_datagen generator
train_ds = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Trimmed_AID_dataset/train',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

# Flow validation images in batches of 32 using validation_datagen generator
validation_ds = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/Trimmed_AID_dataset/val',
    target_size=(256, 256),
    batch_size=32,
    class_mode='categorical'
)

Found 1346 images belonging to 30 classes.
Found 672 images belonging to 30 classes.


In [ ]:
# Build ResNet model
resnet_model = Sequential()
resnet_model.add(ResNet50(include_top=False, weights='imagenet', input_shape=(256, 256, 3)))
resnet_model.add(Flatten())
resnet_model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))  # L2 regularization
resnet_model.add(BatchNormalization())  # Batch normalization layer
resnet_model.add(Dropout(0.5))  # Dropout layer
resnet_model.add(Dense(num_classes, activation='softmax'))


# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

# Compile and fit ResNet model with callbacks
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
resnet_model.fit(train_ds, epochs=10, validation_data=validation_ds, callbacks=[early_stopping, checkpoint])

Epoch 1/10
43/43 [==============================] - 83s 1s/step - loss: 4.2399 - accuracy: 0.2437 - val_loss: 295.8046 - val_accuracy: 0.0298
Epoch 2/10
43/43 [==============================] - 51s 1s/step - loss: 3.8386 - accuracy: 0.3841 - val_loss: 201.3369 - val_accuracy: 0.0298
Epoch 3/10
43/43 [==============================] - 45s 1s/step - loss: 4.6240 - accuracy: 0.2132 - val_loss: 6299.6890 - val_accuracy: 0.0298
Epoch 4/10
43/43 [==============================] - 45s 1s/step - loss: 4.3324 - accuracy: 0.2407 - val_loss: 129.7597 - val_accuracy: 0.0298
Epoch 5/10
43/43 [==============================] - 46s 1s/step - loss: 3.9146 - accuracy: 0.3016 - val_loss: 953.2681 - val_accuracy: 0.0298
Epoch 6/10
43/43 [==============================] - 50s 1s/step - loss: 3.7039 - accuracy: 0.3299 - val_loss: 157.1633 - val_accuracy: 0.0565
Epoch 7/10
43/43 [==============================] - 48s 1s/step - loss: 3.3065 - accuracy: 0.3826 - val_loss: 6.3342 - val_accuracy: 0.0298
Epoch 8

it seems that the training process is unstable, and the model is not converging effectively. The validation loss and accuracy are fluctuating, and the validation accuracy is quite low.

let's try a lower learning rate to see if it helps stabilize the training process and improve convergence. Let's try reducing the learning rate to 0.0001 and see how the model performs

Learning Rate

Lets add some learning rate

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
# Build ResNet model
resnet_model = Sequential()
resnet_model.add(ResNet50(include_top=False, weights='imagenet', input_shape=(256, 256, 3)))
resnet_model.add(Flatten())
resnet_model.add(Dense(256, activation='relu', kernel_regularizer=l2(0.001)))  # L2 regularization
resnet_model.add(BatchNormalization())  # Batch normalization layer
resnet_model.add(Dropout(0.5))  # Dropout layer
resnet_model.add(Dense(num_classes, activation='softmax'))


# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True)

learning_rate = 0.0001

# Compile ResNet model with the specified learning rate
resnet_model.compile(optimizer=Adam(learning_rate=learning_rate),
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

# Fit ResNet model with callbacks
resnet_model.fit(train_ds, epochs=10,
                 validation_data=validation_ds,
                 callbacks=[early_stopping, checkpoint])

Epoch 1/10
43/43 [==============================] - 81s 1s/step - loss: 2.7108 - accuracy: 0.4331 - val_loss: 12.1873 - val_accuracy: 0.0312
Epoch 2/10
43/43 [==============================] - 46s 1s/step - loss: 1.3396 - accuracy: 0.7437 - val_loss: 11.8698 - val_accuracy: 0.0298
Epoch 3/10
43/43 [==============================] - 53s 1s/step - loss: 1.0621 - accuracy: 0.8484 - val_loss: 8.8102 - val_accuracy: 0.0461
Epoch 4/10
43/43 [==============================] - 49s 1s/step - loss: 1.0620 - accuracy: 0.8373 - val_loss: 6.9655 - val_accuracy: 0.0283
Epoch 5/10
43/43 [==============================] - 52s 1s/step - loss: 0.9160 - accuracy: 0.8923 - val_loss: 7.6049 - val_accuracy: 0.0580
Epoch 6/10
43/43 [==============================] - 46s 1s/step - loss: 0.8447 - accuracy: 0.9056 - val_loss: 8.7990 - val_accuracy: 0.0432
Epoch 7/10
43/43 [==============================] - 47s 1s/step - loss: 0.7657 - accuracy: 0.9361 - val_loss: 8.1163 - val_accuracy: 0.0461


Fine Tuning

let's fine-tune the model by unfreezing some layers of the ResNet backbone and allowing them to be updated during training. Here's how you can modify your code to achieve this:

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D

In [ ]:
# Load ResNet50 model without the top classification layer
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

# Freeze all layers in the base model
base_model.trainable = False

# Create a new model on top of the ResNet50 backbone
inputs = tf.keras.Input(shape=(256, 256, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu', kernel_regularizer=l2(0.001))(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
outputs = Dense(num_classes, activation='softmax')(x)
fine_tuned_model = tf.keras.Model(inputs, outputs)

In [ ]:
# Compile the model
learning_rate = 0.0001
fine_tuned_model.compile(optimizer=Adam(learning_rate=learning_rate),
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

# Fine-tune the model by training with unfreezed layers
history = fine_tuned_model.fit(train_ds, epochs=100,
                               validation_data=validation_ds,
                               callbacks=[early_stopping, checkpoint])

Epoch 1/100
43/43 [==============================] - 50s 1s/step - loss: 2.9820 - accuracy: 0.2028 - val_loss: 3.0521 - val_accuracy: 0.1592
Epoch 2/100
43/43 [==============================] - 42s 981ms/step - loss: 2.9177 - accuracy: 0.2080 - val_loss: 3.2823 - val_accuracy: 0.1012
Epoch 3/100
43/43 [==============================] - 40s 938ms/step - loss: 2.8870 - accuracy: 0.2073 - val_loss: 3.2061 - val_accuracy: 0.1801
Epoch 4/100
43/43 [==============================] - 42s 969ms/step - loss: 2.8603 - accuracy: 0.2385 - val_loss: 3.1195 - val_accuracy: 0.2009


In [ ]:
# Compile the model
learning_rate = 0.0001
fine_tuned_model.compile(optimizer=Adam(learning_rate=learning_rate),
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

# Fine-tune the model by training with unfreezed layers
history = fine_tuned_model.fit(train_ds, epochs=50,
                               validation_data=validation_ds)

Epoch 1/50
43/43 [==============================] - 46s 970ms/step - loss: 2.9617 - accuracy: 0.2021 - val_loss: 2.9178 - val_accuracy: 0.2039
Epoch 2/50
43/43 [==============================] - 41s 946ms/step - loss: 2.9045 - accuracy: 0.2110 - val_loss: 2.9692 - val_accuracy: 0.2083
Epoch 3/50
43/43 [==============================] - 41s 956ms/step - loss: 2.8701 - accuracy: 0.2281 - val_loss: 3.0277 - val_accuracy: 0.1414
Epoch 4/50
43/43 [==============================] - 41s 959ms/step - loss: 2.8619 - accuracy: 0.2155 - val_loss: 3.1708 - val_accuracy: 0.1682
Epoch 5/50
43/43 [==============================] - 41s 954ms/step - loss: 2.8543 - accuracy: 0.2080 - val_loss: 3.2324 - val_accuracy: 0.1890
Epoch 6/50
43/43 [==============================] - 42s 980ms/step - loss: 2.7755 - accuracy: 0.2281 - val_loss: 3.3116 - val_accuracy: 0.1518
Epoch 7/50
43/43 [==============================] - 41s 952ms/step - loss: 2.7662 - accuracy: 0.2370 - val_loss: 3.8683 - val_accuracy: 0.1711

KeyboardInterrupt: 